# Muesli Data Analysis

## Pre-Setup

### Environment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid")
sns.set_palette("magma")

### Read in data

In [ ]:
#df_orders = pd.read_excel("./data/Muesli Project raw data.xlsx",sheet_name="Orders",header=1)
#df_campaign = pd.read_excel("./data/Muesli Project raw data.xlsx", sheet_name="Campaign Data")
#df_order_process = pd.read_excel("./data/Muesli Project raw data.xlsx", sheet_name="Order Process Data")
#df_interndata = pd.read_excel("./data/Muesli Project raw data.xlsx", sheet_name="InternData Study")

### Raw dataframes

In [ ]:
df_orders.head(2)

In [ ]:
df_campaign.head(2)

In [ ]:
df_order_process.head(2)

In [ ]:
df_interndata.head(2)

### Data cleaning function

In [ ]:
def data_cleaning(df, df_type):
    
    # make col names lower
    df.columns = df.columns.str.lower()

    
    # orders data
    if df_type == "orders":
        # dropping cols
        df = df.drop(["index", "customer name", "origin channel", "category", "sub-category", "product id", "sales", "quantity", "discount", "profit"],axis=1)
        # dropping duplicates
        df = df.drop_duplicates()
        # renaming values in ship mode
        df["ship mode"] = df["ship mode"].str.replace(" Class","")
        df["ship mode"] = df["ship mode"].str.replace("Second","Standard")
        df["ship mode"] = df["ship mode"].str.replace("First","Express")
    
    # campaign data
    elif df_type == "campaign":
        # dropping cols
        df = df.drop("customer name", axis=1)
        # dropping duplicates
        pass

    # order process data    
    elif df_type == "order_process":
        # dropping cols
        df = df.drop("row id", axis=1)
        # dropping duplicates
        df = df.drop_duplicates()
        # dropping 1 duplicate row for id (scanned on truck twice)
        df = df.drop_duplicates(subset=["order id"], keep = "first")
        # dropping column order date, because 100% match with order date in orders data
        df = df.drop("order date", axis=1)
        # dropping column ship mode, because 100% match with ship mode in orders data (assumption: second class shipping = standard)
        df = df.drop("ship mode", axis=1)


    #intern data
    else:
        # dropping cols
        pass
        # dropping duplicates
        df = df.drop_duplicates()
        # dropping column pickup date, because 100% match with on truck scan date
        df = df.drop("pickup date", axis=1)

    

    return df

### Checks for cleaning function

#### Orders data

In [ ]:
df_orders_1 = data_cleaning(df_orders,"orders")
df_orders_1

In [ ]:
df_orders_1.duplicated().value_counts()

In [ ]:
df_orders_1["order id"].nunique()

In [ ]:
df_orders_1.info()

#### Campaign data

In [ ]:
df_campaign_1 = data_cleaning(df_campaign,"campaign")
df_campaign_1

In [ ]:
df_campaign_1.duplicated().value_counts()

In [ ]:
df_campaign_1["order id"].nunique()

In [ ]:
df_campaign_1.info()

#### Order process data

In [ ]:
df_order_process_1 = data_cleaning(df_order_process,"order_process")
df_order_process_1

In [ ]:
df_order_process_1.duplicated().value_counts()

In [ ]:
df_order_process_1["order id"].duplicated().value_counts()

In [ ]:
df_order_process_1.drop_duplicates("order id")


In [ ]:
duplicates = df_order_process_1[df_order_process_1["order id"].duplicated(keep=False)]
duplicates

In [ ]:
df_order_process_1.info()

#### Intern data

In [ ]:
df_interndata_1 = data_cleaning(df_interndata,"intern")
df_interndata_1

In [ ]:
df_interndata_1.duplicated().value_counts()

In [ ]:
df_interndata_1["order id"].duplicated().value_counts()

In [ ]:
df_interndata_1.info()

#### Truck scan vs intern scan

In [ ]:
#merged_truck = df_order_process_1.merge(df_interndata_1, on="order id", how="outer")
#merged_truck = merged_truck[["order id","on truck scan date","pickup date"]].dropna()
#merged_truck

In [ ]:
#merged_truck["diff"] = merged_truck["on truck scan date"] - merged_truck["pickup date"]
#merged_truck

#### Order dates & shipping methods

In [ ]:
#merged_op = df_orders_1.merge(df_order_process_1, on="order id", how="outer").dropna()

In [ ]:
#merged_op[["order id","order date_x","ship mode_x","order date_y","ship mode_y"]]
#merged_op["date_diff"] = merged_op["order date_x"]-merged_op["order date_y"]
#merged_op["date_diff"].value_counts()

In [ ]:
#merged_op

In [ ]:
#merged_op[["order id","ship mode_x","ship mode_y"]]

In [ ]:
#merged_op["ship mode_x"].value_counts()

In [ ]:
#merged_op["ship mode_y"].value_counts()

In [ ]:
#merged_op["ship mode_x"] = merged_op["ship mode_x"].str.replace(" Class","")
#merged_op["ship mode_y"] = merged_op["ship mode_y"].str.replace(" Processing","")
#merged_op["ship mode_x"] = merged_op["ship mode_x"].str.replace("Second","Standard")
#merged_op["ship mode_x"] = merged_op["ship mode_x"].str.replace("First","Express")

In [ ]:
#merged_op[["order id","ship mode_x","ship mode_y"]]

In [ ]:
#merged_op['match'] = merged_op['ship mode_x'] == merged_op['ship mode_y']
#merged_op['match'].value_counts()

## Cleaned dataframes

In [ ]:
df_orders_cleaned = data_cleaning(df_orders,"orders")
df_campaign_cleaned = data_cleaning(df_campaign,"campaign")
df_order_process_cleaned = data_cleaning(df_order_process,"order_process")
df_interndata_cleaned = data_cleaning(df_interndata,"intern")

In [ ]:
df_orders_cleaned

In [ ]:
df_campaign_cleaned

In [ ]:
df_order_process_cleaned

In [ ]:
df_interndata_cleaned

## Metrics

### o_date_2_processed

In [ ]:
merged_metric_1 = df_orders_cleaned.merge(df_interndata_cleaned, on="order id", how="outer").dropna()
merged_metric_1

In [ ]:
merged_metric_1["date_diff"] = merged_metric_1["ready to ship date"]-merged_metric_1["order date"]
merged_metric_1

In [ ]:
merged_metric_1.info()

In [ ]:
merged_metric_1["date_diff_days"] = merged_metric_1["date_diff"].dt.days

In [ ]:
merged_metric_1.info()

In [ ]:
# 1. Distribution of date_diff
plt.figure(figsize=(8,5))
plt.hist(merged_metric_1["date_diff_days"], edgecolor="black")
plt.title("Distribution of date difference")
plt.xlabel("Days between order and ready to ship")
plt.ylabel("Frequency")
plt.show()

### o_processes_2_truck

#### Data

In [ ]:
df_processed = df_interndata_cleaned.copy()
df_processed["ready to ship date_weekday"] = df_processed["ready to ship date"].dt.dayofweek
df_processed

In [ ]:
df_processed["ready to ship date_weekday"].value_counts()

In [ ]:
# ready to ship date only Mo - Fr checked!

In [ ]:
df_processed_x_truck = df_order_process_cleaned.merge(df_processed, on="order id",how="inner")
df_processed_x_truck

In [ ]:
df_processed_x_truck_smode = df_processed_x_truck.merge(df_orders_cleaned, on="order id", how="inner")
df_processed_x_truck_smode = df_processed_x_truck_smode[["order id", "on truck scan date", "ready to ship date", "ready to ship date_weekday","ship mode","order date"]]
df_processed_x_truck_smode["o_processed_2_truck_actual"] = df_processed_x_truck_smethod["on truck scan date"] - df_processed_x_truck_smethod["ready to ship date"]
df_processed_x_truck_smode["o_processed_2_truck_actual"] = df_processed_x_truck_smode["o_processed_2_truck_actual"].dt.days
df_processed_x_truck_smode

In [ ]:
df_processed_x_truck_smode["order date_weekday"] = df_processed_x_truck_smode["order date"].dt.day_name()
df_processed_x_truck_smode

In [ ]:
df_processed_x_truck_smode.groupby("ship mode")[["o_processed_2_truck_actual"]].agg(["mean","max","min"])

In [ ]:
graph_1 = df_processed_x_truck_smode.groupby("ship mode")[["o_processed_2_truck_actual"]].mean().reset_index()
graph_1

In [ ]:
graph_1["o_processed_2_truck_plan"] = {0:0,1:1}
graph_1

In [ ]:
# format for plot
graph_1 = pd.melt(graph_1,
    id_vars=["ship mode"],
    value_vars=["o_processed_2_truck_actual", "o_processed_2_truck_plan"],
    var_name="data type",
    value_name="days")

# renaming data type
graph_1["data type"] = graph_1["data type"].map({
    "o_processed_2_truck_actual": "Actual",
    "o_processed_2_truck_plan": "Plan"})

graph_1

In [ ]:
graph_2 = df_processed_x_truck_smode.groupby(["ship mode","order date_weekday"])[["o_processed_2_truck_actual"]].mean().reset_index()
graph_2

In [ ]:
# no sunday for express?

In [ ]:
graph_2["identifyer"]=graph_2["ship mode"]+graph_2["order date_weekday"]
graph_2

In [ ]:
plan_values_dict = {"ExpressMonday":1,
                    "ExpressTuesday":0,
                    "ExpressWednesday":1,
                    "ExpressThursday":0,
                    "ExpressFriday":0,
                    "ExpressSaturday":1,
                    "ExpressSunday":1,
                    "StandardMonday":1,
                    "StandardTuesday":2,
                    "StandardWednesday":1,
                    "StandardThursday":3,
                    "StandardFriday":2,
                    "StandardSaturday":1,
                    "StandardSunday":1
                    }

In [ ]:
graph_2["o_processed_2_truck_plan"] = graph_2["identifyer"].map(plan_values_dict)
graph_2 = graph_2.drop("identifyer",axis=1)
graph_2

In [ ]:
# format for plot
graph_2 = pd.melt(graph_2,
    id_vars=["ship mode","order date_weekday"],
    value_vars=["o_processed_2_truck_actual", "o_processed_2_truck_plan"],
    var_name="data type",
    value_name="days")

# renaming data type
graph_2["data type"] = graph_2["data type"].map({
    "o_processed_2_truck_actual": "Actual",
    "o_processed_2_truck_plan": "Plan"})

graph_2

In [ ]:
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

graph_2["order date_weekday"] = pd.Categorical(graph_2["order date_weekday"],categories=weekday_order, ordered=True)
graph_2

In [ ]:
graph_2_express = graph_2[graph_2["ship mode"]=="Express"].sort_values("order date_weekday")
graph_2_standard = graph_2[graph_2["ship mode"]=="Standard"].sort_values("order date_weekday")

graph_2_express

#### Plots

In [ ]:
graph = graph_1

plt.figure(figsize=(10, 7))

ax = sns.barplot(data=graph, 
             x= "ship mode",
             y = "days",
             hue="data type",
             hue_order=["Actual","Plan"],
             width=bar_width
            )
ax.set_ylim(-0.1, 3.2)
plt.title('Order ready for shipment 2 leaving warehouse', fontsize=16,pad=20)
plt.xlabel("Shipping type")
plt.ylabel("Days")
plt.legend()

bar_width = 0.7
offset = (bar_width / 4) # This calculates the offset needed to center on the 'Actual' bar

# Calculate and display the uplift (difference) as text above the bars
# Iterate through each ship mode ('Express' and 'Standard')
for i, ship_mode in enumerate(graph["ship mode"].unique()):
    # Get the 'Actual' and 'Plan' values for the current ship mode
    actual_days = graph[(graph["ship mode"] == ship_mode) & (graph["data type"] == "Actual")]["days"].iloc[0]
    plan_days = graph[(graph["ship mode"] == ship_mode) & (graph["data type"] == "Plan")]["days"].iloc[0]

    uplift = actual_days - plan_days
    
    # X-POSITION: Centered over the 'Actual' bar.
    # The first bar group (Express, i=0) is at x=0. The second (Standard, i=1) is at x=1.
    # The 'Actual' bar is typically slightly to the left of the center point (i - offset).
    x_pos = i - offset
    # y-position: slightly above the higher bar for the annotation
    y_pos = actual_days +0.05 # Adjust vertical position as needed

    # Determine text color (Green for improvement (lower days), Red for regression (higher days))
    color = "black"
    
    # Format the uplift text (e.g., "+0.50 days" or "-0.25 days")
    uplift_text = f"{uplift:+.1f} d." # Using + to explicitly show positive or negative

    # Add the uplift text
    ax.text(x_pos, y_pos, uplift_text,
            color=color, ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xticks()
plt.xticks()
plt.tight_layout()
plt.show(),

In [ ]:
graph = graph_2_standard

plt.figure(figsize=(10, 7))

ax = sns.barplot(data=graph, 
             x= "order date_weekday",
             y = "days",
             hue="data type",
             hue_order=["Actual","Plan"],
             width=bar_width,
             errorbar=None
            )
ax.set_ylim(-0.1, 3.2)
plt.title('*Standard* order ready for shipment 2 leaving warehouse', fontsize=16,pad=20)
plt.xlabel("Order weekday")
plt.ylabel("Days")
plt.legend()

bar_width = 0.7
offset = (bar_width / 4) # This calculates the offset needed to center on the 'Actual' bar


# Calculate and display the uplift (difference) as text above the bars
# Iterate through each unique WEEKDAY in the data
for i, weekday in enumerate(graph["order date_weekday"].unique()):
    # Get the 'Actual' and 'Plan' values for the current WEEKDAY
    
    # Filter the graph DataFrame by the current weekday
    weekday_data = graph[graph["order date_weekday"] == weekday]
    
    # Use the filtered data to extract the specific Actual and Plan values
    actual_days = weekday_data[weekday_data["data type"] == "Actual"]["days"].iloc[0]
    plan_days = weekday_data[weekday_data["data type"] == "Plan"]["days"].iloc[0]

    uplift = actual_days - plan_days
    
    # X-POSITION: Centered over the 'Actual' bar.
    # The first bar group (Express, i=0) is at x=0. The second (Standard, i=1) is at x=1.
    # The 'Actual' bar is typically slightly to the left of the center point (i - offset).
    x_pos = i - offset
    # y-position: slightly above the higher bar for the annotation
    y_pos = actual_days +0.05 # Adjust vertical position as needed

    # Determine text color (Green for improvement (lower days), Red for regression (higher days))
    color = "black"
    
    # Format the uplift text (e.g., "+0.50 days" or "-0.25 days")
    uplift_text = f"{uplift:+.1f} d." # Using + to explicitly show positive or negative

    # Add the uplift text
    ax.text(x_pos, y_pos, uplift_text,
            color=color, ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xticks()
plt.xticks()
plt.tight_layout()
plt.show(),

In [ ]:
graph = graph_2_express

plt.figure(figsize=(10, 7))

ax = sns.barplot(data=graph, 
             x= "order date_weekday",
             y = "days",
             hue="data type",
             hue_order=["Actual","Plan"],
             width=bar_width,
             errorbar=None
            )
ax.set_ylim(-0.1, 3.2)
plt.title('*Express* order ready for shipment 2 leaving warehouse', fontsize=16,pad=20)
plt.xlabel("Order weekday")
plt.ylabel("Days")
plt.legend()

bar_width = 0.7
offset = (bar_width / 4) # This calculates the offset needed to center on the 'Actual' bar


# Calculate and display the uplift (difference) as text above the bars
# Iterate through each unique WEEKDAY in the data
for i, weekday in enumerate(graph["order date_weekday"].unique()):
    # Get the 'Actual' and 'Plan' values for the current WEEKDAY
    
    # Filter the graph DataFrame by the current weekday
    weekday_data = graph[graph["order date_weekday"] == weekday]
    
    # Use the filtered data to extract the specific Actual and Plan values
    actual_days = weekday_data[weekday_data["data type"] == "Actual"]["days"].iloc[0]
    plan_days = weekday_data[weekday_data["data type"] == "Plan"]["days"].iloc[0]

    uplift = actual_days - plan_days
    
    # X-POSITION: Centered over the 'Actual' bar.
    # The first bar group (Express, i=0) is at x=0. The second (Standard, i=1) is at x=1.
    # The 'Actual' bar is typically slightly to the left of the center point (i - offset).
    x_pos = i - offset
    # y-position: slightly above the higher bar for the annotation
    y_pos = actual_days +0.05 # Adjust vertical position as needed

    # Determine text color (Green for improvement (lower days), Red for regression (higher days))
    color = "black"
    
    # Format the uplift text (e.g., "+0.50 days" or "-0.25 days")
    uplift_text = f"{uplift:+.1f} d." # Using + to explicitly show positive or negative

    # Add the uplift text
    ax.text(x_pos, y_pos, uplift_text,
            color=color, ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xticks()
plt.xticks()
plt.tight_layout()
plt.show(),

In [ ]:
graph_2_express

### o_truck_2_delivered

### o_date_2_delivered

### o_processed_2_delivered

#### Data

In [ ]:
df_processed_x_delivered = df_campaign_cleaned.merge(df_interndata_cleaned, on="order id",how="inner")
df_processed_x_delivered

#### Plots

In [ ]:
#not enough data